In [1]:
import numpy as np
import scipy
from scipy import stats
from numpy       import pi, sqrt, log, exp
from scipy.stats import norm
from scipy.stats import qmc
import scipy.linalg as la
from numpy import linalg as LA

In [2]:
for i in range(2):
    N = 2**20
    r = 0.05
    sigma = 0.2
    T = 1
    K = 100
    S0 = 100
    Sigma = (sigma**2)*T*(np.identity(5)+0.1*(np.ones(5)-np.identity(5)))
    if i == 0:
        print('Cholesky factorisation: ')
        L = la.cholesky(Sigma,lower=True)
    else:
        print(' ')
        print('PCA factorisation: ')
        w, V = LA.eig(Sigma)
        L = np.matmul(V,sqrt(np.diag(w)))
        
    #doing standard MC
    U = np.random.rand(5,N)
    #U is a 5xN matrix containing iid U[0,1) RVs as entries
    Y = norm.ppf(U)
    #ppf is the inverse of the cdf
    S = S0*exp((r-0.5*(sigma**2))*T+np.matmul(L,Y))
    F = exp(-r*T)*np.maximum(0,0.2*np.sum(S,0)-K)
    sum1 = np.sum(F)
    sum2 = np.sum(np.power(F,2))
    val = sum1/N
    var = sum2/N-val**2
    err_bound = 3*sqrt(var/N)
    print('MC_val = ',val)
    print('MC_err_bound = ',err_bound)
    
    #Latin Hypercube
    M = 1024
    N = 1024
    #M-number of strata, N-number of Latin Hypercube samples
    sum1 = 0
    sum2 = 0
    m = np.zeros(shape=(5,M))
    for n in range(N):
        for d in range(5):
            m[d,:] = np.random.permutation(M)
            #Latin Hypercube locations
        U = (m+np.random.rand(5,M))/M
        #m instead of m-1 as Python starts at 0, not 1
        #random location within each cube
        Y = norm.ppf(U)
        S = S0*exp((r-0.5*(sigma**2))*T+np.matmul(L,Y))
        F = exp(-r*T)*np.maximum(0,0.2*np.sum(S,0)-K)
        sum1 = sum1+np.sum(F)/M
        sum2 = sum2+(np.sum(F)/M)**2
    val = sum1/N
    var = sum2/N-val**2
    err_bound = 3*sqrt(var/N)
    print('LH_val = ',val)
    print('LH_err_bound = ',err_bound)
    
    
    #now QMC
    Ntot = 2**20
    #number of samples to take
    N = 2**4
    #number of sets
    M = Ntot/N
    M_base2 = 16
    #number of QMC points
    sum1 = 0
    sum2 = 0
    
    for n in range(N):
        #simulate for each set of Sobol points
        Ps = qmc.Sobol(d=5,scramble=True)
        #dimension 5, default is Owen scrambling
        U = np.transpose(Ps.random_base2(M_base2))
        #generate a set of M Sobol points
        #random_base2 is safer ro use
        Y = norm.ppf(U)
        S = S0*exp((r-0.5*(sigma**2))*T+np.matmul(L,Y))
        F = exp(-r*T)*np.maximum(0,0.2*np.sum(S,0)-K)
        sum1 = sum1+np.sum(F)/M
        sum2 = sum2+(np.sum(F)/M)**2
    
    
    val = sum1/N
    var = sum2/N-val**2
    err_bound = 3*sqrt(var/N)
    print('QMC_val = ',val)
    print('QMC_err_bound = ',err_bound)
        

Cholesky factorisation: 
MC_val =  7.016432167555545
MC_err_bound =  0.023883557215689466
LH_val =  7.023351139592524
LH_err_bound =  0.007682797628028239
QMC_val =  7.023071847497049
QMC_err_bound =  0.0007582557551233198
 
PCA factorisation: 
MC_val =  7.021465806459481
MC_err_bound =  0.02389314591715821
LH_val =  7.023134571443001
LH_err_bound =  0.001467077157090056
QMC_val =  7.022859432761037
QMC_err_bound =  8.683345314737322e-05
